## TextBlob 라이브러리를 이용한 감정분석(긍/부정)
- 학습 데이터: ratings_train.txt

In [3]:
import pandas as pd
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier

In [7]:
train_data = pd.read_table("ratings_train.txt")
train_data

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [8]:
pos_df = train_data[train_data["label"] == 1].head(1000)
neg_df = train_data[train_data["label"] == 0].head(1000)
train_data1 = pd.concat([pos_df, neg_df])
train_data1 = train_data1.sort_values("id")
train_data1 = train_data1.reset_index(drop=True)
train_data1

,id,document,label
0,11633,하하하 그럼 전무송이 괴노인이겠네 ㅋㅋ 보고싶은 영화군,1
1,14513,정말 비디오마저 돈주고 보고 아깝다..최악...,0
2,20186,[볼륨을 높여라]의 재치는 어디에..,0
3,21452,단순무식한 각본 하나가 명감독 명배우들의 작품을 망쳐놨다..,0
4,23653,B급을 지향하는 C급 작품.,0
...,...,...,...
1995,10273782,감독님들 고은님 쓰면 영화안봅니다 .,0
1996,10274403,OOO영화 뭘전달하려는지모르겠네 오글거리고,0
1997,10275296,미달이는.. 연예계를 바라보는 태도를 고쳐야 한다.,0
1998,10276348,인간들아 욕 좀 그만해라 금발의 여 주인공 쌕시하고 예뻐서 시종 눈을 뗄 수 없는 ...,1


In [9]:
# NaN 값 확인
train_data1.isna().sum()

id          0
document    0
label       0
dtype: int64

In [10]:
# test 데이터 500개로 정의하기
test_data1=pd.concat([train_data[train_data['label']==1].tail(250),
                       train_data[train_data['label']==0].tail(250)])
test_data1=test_data1.sort_values('id')
test_data1=test_data1.reset_index(drop=True)
test_data1

,id,document,label
0,5793,진실을 말하라는 교훈을 주는 아이들용 가족영화..나이많으면 잼없음,0
1,15460,"최강 트라이건, 명작 트라이건",1
2,30328,보다말아뿌따 제길,0
3,36734,정말 재미있다. 특히 원영의의 연기가 인상적이다. 노래도 좋다.,1
4,37128,호러 매니아들로서 보면 후회하게 하는 영화 고정관념을 버리고 사심없이..,0
...,...,...,...
495,10253254,"진심 이영화 진짜 역대급에 들정도로 3류 미국 성인영화 내용에 진지함 한스푼 추가,...",0
496,10260086,호러블 어보미네이션 오브 멘카인드.이게 존재하는게 소름....이걸로 상받은것도 소름...,0
497,10260970,루카스 멋있어요. 영화좀 골라서 찍으세요 ㅜㅜ,0
498,10271179,한국 판타지 영화에 한 획을 그은 영화,0


In [11]:
# NaN 값 확인
test_data1.isna().sum()

id          0
document    0
label       0
dtype: int64

In [12]:
# 튜플 구조로 변경
def tup_che(df_t):
    tr_te =[]
    for i in range(len(df_t)):
        tr_te.append((df_t.iloc[i, 0], df_t.iloc[i, 1]))
    return tr_te

In [13]:
del train_data1['id'], test_data1['id']

train = tup_che(train_data1)
test = tup_che(test_data1)

train[:3]

[('하하하 그럼 전무송이 괴노인이겠네 ㅋㅋ 보고싶은 영화군', 1),
 ('정말 비디오마저 돈주고 보고 아깝다..최악...', 0),
 ('[볼륨을 높여라]의 재치는 어디에..', 0)]

In [14]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
pos_neg = NaiveBayesClassifier(train)
test_acc = pos_neg.accuracy(test)

print(f"정확도: {test_acc}")

정확도: 0.712


In [22]:
rev_df = pd.read_csv('nav_move_review.csv')
rev_df

,제목,감상평,평점,날짜
0,신비한 동물들과 덤블도어의 비밀,'역대 최악으로 재미가 없음. ',2,22.04.20
1,레드 로켓,'인간의 추악함과 이기심 그리고 아메리칸드림을 비웃는 웃지 못할 코믹 영화. B급...,9,22.04.20
2,앰뷸런스,'스토리 재밌음.근데 토할뻔함. (영화 내내 차 안에서 스토리진행 매우 답답 + ...,6,22.04.20
3,더 배트맨,'1.pc묻음 2.평레기들 질질쌈',1,22.04.20
4,"안녕, 나의 소울메이트",'내인생 최고의 영화^^',10,22.04.20
...,...,...,...,...
995,제8일의 밤,'김유정이 존나 이쁨',8,22.04.19
996,더 배트맨,'일단 영화 전체적인 분위기만으로 베트맨에 조금이라도 관심이 있는 팬들은 만족스러...,9,22.04.19
997,더 배트맨,'복수에서 희망으로...',10,22.04.19
998,이터널 선샤인,'전체적으로 산만해서 처음오프닝의 잔잔함이 옅어지며 감정이입이 어려워지는 아쉬움이있다',6,22.04.19


In [23]:
posneg = pos_neg.classify(rev_df.loc[499, "감상평"])
posneg

1

In [24]:
pos_neg_lst=[]

for txt in rev_df['감상평']:
    pos_neg_lst.append(pos_neg.classify(txt))

rev_df['긍부정']=pos_neg_lst
rev_df

,제목,감상평,평점,날짜,긍부정
0,신비한 동물들과 덤블도어의 비밀,'역대 최악으로 재미가 없음. ',2,22.04.20,0
1,레드 로켓,'인간의 추악함과 이기심 그리고 아메리칸드림을 비웃는 웃지 못할 코믹 영화. B급...,9,22.04.20,0
2,앰뷸런스,'스토리 재밌음.근데 토할뻔함. (영화 내내 차 안에서 스토리진행 매우 답답 + ...,6,22.04.20,0
3,더 배트맨,'1.pc묻음 2.평레기들 질질쌈',1,22.04.20,0
4,"안녕, 나의 소울메이트",'내인생 최고의 영화^^',10,22.04.20,1
...,...,...,...,...,...
995,제8일의 밤,'김유정이 존나 이쁨',8,22.04.19,0
996,더 배트맨,'일단 영화 전체적인 분위기만으로 베트맨에 조금이라도 관심이 있는 팬들은 만족스러...,9,22.04.19,1
997,더 배트맨,'복수에서 희망으로...',10,22.04.19,0
998,이터널 선샤인,'전체적으로 산만해서 처음오프닝의 잔잔함이 옅어지며 감정이입이 어려워지는 아쉬움이있다',6,22.04.19,0


In [26]:
rev_df[rev_df["평점"]>=8].groupby("긍부정")[["평점"]].count()

,평점
긍부정,
0,215
1,335


In [28]:
rev_df1 = pd.read_csv('nav_movie_review1.csv')
rev_df1

,제목,평점,리뷰,날짜
0,불도저에 탄 소녀,6,연기가 아니었다면 살리지 못했을 영화,22.04.20
1,먼 훗날 우리,0,중국영화를 다시 보게한 수작!그옛날 나의 청춘을 생각하게 하는 그런...유미야 보고...,22.04.20
2,해적: 도깨비 깃발,6,에이 좀 재미없었다 배우들이 조금 아까워요말투 어색 ㅠ,22.04.20
3,더 배트맨,5,다른건 모르겠고 여주가 너무 못생김..,22.04.20
4,로스트 시티,5,너흰 나에게 똥을 줬어 아주 큰 똥을 브래드피트 땜시 그나마 5점 줌,22.04.20
...,...,...,...,...
495,셰이프 오브 워터: 사랑의 모양,0,"간만에 아주 몰입해서 재미있게 봤습니다. 색감이나 소품 등 눈이 즐겁고, 사랑이 이...",22.04.19
496,야차,2,와우 이건 뭐죠? 이것이 21세기 넷플릭스 영화가 맞습니까ㅋㅋ 평점은 0점인데 넷플...,22.04.19
497,해적: 도깨비 깃발,8,뭔가 좀 어설픈 영상도 있지만;;;뭐 그냥 나름 볼만했어요 ?한효주 연기 진짜 못...,22.04.19
498,자산어보,0,흑백임에도 불구하고 전혀 지루하지 않았다. 모두가 꼭 한 번은 봤으면 하는 영화. ...,22.04.19


In [30]:
posneg = pos_neg.classify(rev_df1.loc[499, "리뷰"])
posneg

1

In [31]:
pos_neg_lst=[]

for txt in rev_df1['리뷰']:
    pos_neg_lst.append(pos_neg.classify(txt))

rev_df1['긍부정']=pos_neg_lst
rev_df1

TypeError: ignored